In [1]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch

2023-12-07 22:28:16.974810: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 22:28:17.935015: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-07 22:28:17.935108: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-12-07 22:28:17.935120: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
class TextDataset(Dataset):
    def __init__(self, text_file, tokenizer):
        self.tokenizer = tokenizer
        self.data = []
        with open(text_file, 'r') as file:
            for line in file:
                self.data.append(tokenizer(line, truncation=True, padding='max_length', max_length=128))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
dataset = TextDataset('fr_part_1.txt', tokenizer)
dataloader = DataLoader(dataset, batch_size=16)

In [3]:
model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 5
for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {total_loss / len(dataloader)}")

AttributeError: 'list' object has no attribute 'to'

In [ ]:
# Supposons que "sentence" est la phrase que vous voulez tester, avec un token masqué
sentence = "J'aime le <mask>."

# Préparation de l'entrée
input = tokenizer.encode(sentence, return_tensors="pt").to(device)

# Prédiction du token masqué
output = model(input)
prediction = torch.argmax(output.logits, dim=2)

# Conversion du token prédit en mot
predicted_token = tokenizer.convert_ids_to_tokens(prediction[0].tolist())
print(predicted_token)